In [1]:
!pip install keras_tuner --quiet

import os, warnings

warnings.filterwarnings('ignore')

import tensorflow as tf

import keras_tuner as kt

from tensorflow.keras import layers, models, datasets, optimizers

In [2]:
import numpy as np

(train_image, train_label), (test_image, test_label) = datasets.mnist.load_data()

train_image = tf.cast(train_image, tf.float32) / 255.0

In [3]:
test_image = tf.cast(test_image, tf.float32) / 255.0

np.min(test_image[0]), np.max(test_image[0])

(0.0, 1.0)

In [4]:
from tensorflow.keras.models import Sequential

input_shape = (28, 28)

model = Sequential([
  # Create Simple Model
  layers.Flatten(input_shape=input_shape),
  layers.Dense(48, activation='relu'),
  layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

hist = model.fit(train_image, train_label, validation_data=(test_image, test_label), epochs=5, verbose=False)

In [5]:
model.evaluate(train_image, train_label, verbose=False)

[0.05973245948553085, 0.9821166396141052]

In [6]:
model.evaluate(test_image, test_label, verbose=False)

[0.08800793439149857, 0.9731000065803528]

In [ ]:
loss = 'sparse_categorical_crossentropy'

def model_builder(hp):
  """
  Model Builder
  """
  # Define Model
  model = Sequential()
  # Model Creation
  unit_option = hp.Int('units', min_value=16, max_value=64, step=16)
  model.add(layers.Flatten(input_shape=input_shape))
  model.add(layers.Dense(unit_option, activation='relu'))
  model.add(layers.Dense(10, activation='softmax'))
  # Model Compile
  lr_option = hp.Choice('learning_rate', [0.001, 0.01, 0.005])
  optim = optimizers.Adam(learning_rate=lr_option)
  model.compile(optimizer=optim, loss=loss, metrics=['accuracy'])
  return model

project = 'intro_to_kt'

dirt = 'tunerfolder'; target = 'val_accuracy'

tuner = kt.Hyperband(model_builder, objective=target, max_epochs=8, directory=dirt, project_name=project)

In [8]:
tuner.search(train_image, train_label, epochs=5, validation_data=(test_image, test_label))

In [9]:
optimal = tuner.get_best_hyperparameters()[0]

optimal.get('units')

64

In [10]:
optimal.get('learning_rate')

0.001

In [ ]:
optimal_model = tuner.hypermodel.build(optimal)

hist = optimal_model.fit(train_image, train_label, epochs=5, validation_data=(test_image, test_label), verbose=False)

In [ ]:
optimal_model.evaluate(train_image, train_label, verbose=False)

In [ ]:
optimal_model.evaluate(test_image, test_label, verbose=False)